## Pokročilé transformace dat v Pandas

- Pandas dataframe je možné převádět z dlouhých dat na široká (tj. převod řádků na sloupce) či ze širokých na dlouhá (tj. převod sloupců na řádky).
- V rámci těchto transformací je u některých funkcí možné provádět agregaci hodnot.
- Vhodnost použití konkrétní funkce ovlivňují:
    - Struktura dat (chci pracovat s indexi / víceúrovňovými indexi či zůstat u práce se sloupci).
    - Existence požadavku na agregaci dat (je třeba / není třeba).

Úvod do transformací v dokumentaci Pandas: https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#reshaping-stacking


### Pivot

- **Kdy použít:** Když chcete přeformátovat data z dlouhého formátu do širokého bez agregace, typicky pro situace, kde máte jasné mapování mezi indexem, sloupci a hodnotami, a každá kombinace indexu a sloupce je unikátní.
- **Důležité parametry:**
  - `index`: Sloupec nebo sloupce, které se použijí jako nový index.
  - `columns`: Sloupec nebo sloupce, které se stanou novými sloupci datového rámce.
  - `values`: Sloupec nebo sloupce, které se použijí pro vyplnění hodnot v novém datovém rámci.

### Pivot Table

- **Kdy použít:** Pro přetváření dat z dlouhého formátu do širokého s možností agregace, vhodné pro situace, kde se v datech mohou vyskytovat duplicitní kombinace indexu a sloupce.
- **Důležité parametry:**
  - `index`: Sloupec nebo sloupce, které se použijí jako nový index.
  - `columns`: Sloupec nebo sloupce, které se stanou novými sloupci datového rámce.
  - `values`: Sloupec nebo sloupce, které se použijí pro vyplnění hodnot v novém datovém rámci.
  - `aggfunc`: Funkce nebo seznam funkcí pro agregaci dat, např. 'sum', 'mean'.

### Stack

- **Kdy použít:** Když potřebujete převést sloupce na řádky, často používáno pro přeformátování širokého formátu dat do dlouhého, zejména v případě datových rámců s víceúrovňovými sloupci.
- **Důležité parametry:**
  - `level`: Úroveň nebo úrovně indexů sloupců, které chcete "stohovat". Výchozí je poslední úroveň.

### Unstack

- **Kdy použít:** Pro převod úrovní indexu řádků na sloupce, čímž se z dlouhých dat stávají široká. Užitečné pro práci s datovými rámci, které mají víceúrovňový index.
- **Důležité parametry:**
  - `level`: Specifická úroveň indexu, kterou chcete převést na sloupce. Výchozí je poslední úroveň.

### Crosstab

- **Kdy použít:** Pro výpočet kontingenční tabulky, která počítá frekvenci různých kombinací ve dvou nebo více sloupcích. Ideální pro srovnávací studie a statistickou analýzu.
- **Důležité parametry:**
  - `index`: Sloupec nebo sloupce, které se použijí jako řádky kontingenční tabulky.
  - `columns`: Sloupec nebo sloupce, které se stanou sloupci kontingenční tabulky.
  - `values`: Volitelně, sloupec, jehož hodnoty budou agregovány.
  - `aggfunc`: Funkce pro agregaci, použije se, pokud je zadán `values`.

### Melt

- **Kdy použít:** Pro transformaci širokého formátu dat do dlouhého tím, že "rozpustíte" několik sloupců do dvou: jednoho pro názvy proměnných a druhého pro jejich hodnoty. Vhodné pro resetování struktury datového rámce k další analýze nebo vizualizaci, když potřebujete pracovat s dlouhým formátem dat.
- **Důležité parametry:**
  - `id_vars`: Sloupec nebo sloupce, které mají zůstat jako identifikátory; tyto sloupce nebudou transformovány.
  - `value_vars`: Volitelně, specifické sloupce, které mají být rozpuštěny. Pokud není zadáno, všechny ostatní sloupce nezahrnuté v `id_vars` budou rozpuštěny.
  - `var_name`: Název nového sloupce, který bude obsahovat názvy původních sloupců. Výchozí hodnota je 'variable'.
  - `value_name`: Název nového sloupce, který bude obsahovat hodnoty odpovídajících sloupců. Výchozí hodnota je 'value'.


 

| Funkce        | Přetváří dlouhá data na široká | Přetváří široká data na dlouhá | Podporuje agregaci | Pracuje s víceúrovňovými indexy | Umožňuje specifikovat hodnoty/agregace |
|---------------|--------------------------------|--------------------------------|--------------------|----------------------------------|-----------------------------------------|
| `pivot`       | Ano                           | Ne                             | Ne                 | Ne                               | Ano                                    |
| `pivot_table` | Ano                           | Ne                             | Ano                | Ano                              | Ano, včetně výběru agregační funkce    |
| `stack`       | Ne                            | Ano                            | Ne                 | Ano                              | Ne                                     |
| `unstack`     | Ne                            | Ano (částečně)                 | Ne                 | Ano                              | Ne                                     |
| `crosstab`    | Ano                           | Ne                             | Ano                | Ne                               | Ano, s možností použít různé agregace  |
| `melt`        | Ne                            | Ano                            | Ne                 | Ne                               | Ne, ale umožňuje specifikovat proměnné |


In [1]:
import pandas as pd 

df = pd.read_csv('https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv',delimiter=';')
df['DATE'] = pd.to_datetime(df['DATE'])
df['ROK'] = df['DATE'].dt.year

df = df.loc[(df['NAME'].isin(['RUZYNE','MOSNOV'])) & (df['ROK'].isin([2010,2011,2012]))]

df = df.groupby(['ROK','NAME']).agg({'TMAX':'max'}).reset_index()


df

ROK    NAME  TMAX
0  2010  MOSNOV  35.9
1  2010  RUZYNE  34.6
2  2011  MOSNOV  33.3
3  2011  RUZYNE  30.1
4  2012  MOSNOV  34.2
5  2012  RUZYNE  37.4

In [12]:
# Transformace za použití pivot

df.pivot(index='ROK',columns='NAME',values='TMAX')



NAME  MOSNOV  RUZYNE
ROK                 
2010    35.9    34.6
2011    33.3    30.1
2012    34.2    37.4

In [11]:
df.pivot_table(index='ROK',columns='NAME',values='TMAX',aggfunc='max')


NAME  MOSNOV  RUZYNE
ROK                 
2010    35.9    34.6
2011    33.3    30.1
2012    34.2    37.4

In [22]:
df_unstack = df.set_index(['ROK','NAME'])
df_unstack = df_unstack.stack()
df_unstack = df_unstack.unstack(level=1)
df_unstack

NAME       MOSNOV  RUZYNE
ROK                      
2010 TMAX    35.9    34.6
2011 TMAX    33.3    30.1
2012 TMAX    34.2    37.4